In [ ]:
import nltk
from nltk.corpus import stopwords

# Baixar recursos do NLTK
nltk.download('punkt_tab')
nltk.download('stopwords')

In [ ]:
import PyPDF2

# Função para extrair texto de PDFs
def pdf_para_txt(caminho_pdf):
    with open(caminho_pdf, "rb") as f:
        leitor = PyPDF2.PdfReader(f)
        texto = ""
        for pagina in range(len(leitor.pages)):
            texto += leitor.pages[pagina].extract_text()
    return texto

# Diretórios com os PDFs
diretorios = {
    'poesia': 'pdfs/poesia/',
    'prosa': 'pdfs/prosa/',
    'jornalismo': 'pdfs/jornalismo/'
}

# Função para limpar e remover stopwords
def limpar_texto(texto):
    texto = texto.replace("\n", " ")
    stop_words = set(stopwords.words("english"))
    palavras = nltk.word_tokenize(texto.lower())
    palavras_limpa = [palavra for palavra in palavras if palavra.isalnum() and palavra not in stop_words]
    return " ".join(palavras_limpa)


In [ ]:
import os

# Extraindo textos e gerando classes
textos = []
classes = []

for classe, caminho in diretorios.items():
    for arquivo in os.listdir(caminho):
        if arquivo.endswith(".pdf"):
            texto = pdf_para_txt(os.path.join(caminho, arquivo))
            texto_limpo = limpar_texto(texto)
            textos.append(texto_limpo)
            classes.append(classe)

In [128]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Criando a matriz Bag of Words
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(textos)

print("Matriz BoW: ", X.toarray())
print("Vocabulário: ", vectorizer.get_feature_names_out())
print("Classes: ", classes)

Matriz BoW:  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Vocabulário:  ['01' '02' '03' ... '⁷sir' '⁸itisreservedtotheendoftheday' '⁹it']
Classes:  ['poesia' 'poesia' 'poesia' 'poesia' 'poesia' 'poesia' 'poesia' 'poesia'
 'poesia' 'poesia' 'poesia' 'prosa' 'prosa' 'prosa' 'prosa' 'prosa'
 'prosa' 'prosa' 'prosa' 'prosa' 'prosa' 'prosa' 'jornalismo' 'jornalismo'
 'jornalismo' 'jornalismo' 'jornalismo' 'jornalismo' 'jornalismo'
 'jornalismo' 'jornalismo' 'jornalismo' 'jornalismo']


In [123]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

# Criando o modelo de árvore de decisão
modelo = DecisionTreeClassifier()

# Configurando a validação cruzada estratificada
kf = StratifiedKFold(n_splits=10, shuffle=True)

# Listas para armazenar as métricas
acuracias = []
f1_scores = []

# Validação cruzada
for train_index, test_index in kf.split(X, classes):
    # Dividindo os dados em treino e teste para a rodada atual
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = [classes[i] for i in train_index], [classes[i] for i in test_index]
    
    # Treinando o modelo
    modelo.fit(X_train, y_train)
    
    # Fazendo previsões
    y_pred = modelo.predict(X_test)
    
    # Calculando métricas
    acuracia = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    # Armazenando os resultados
    acuracias.append(acuracia)
    f1_scores.append(f1)

# Calcular média e desvio padrão das métricas
acuracia_media, acuracia_desvio = np.mean(acuracias), np.std(acuracias)
f1_media, f1_desvio = np.mean(f1_scores), np.std(f1_scores)

# Exibir resultados
print("Resultados por Fold:")
for i, (acc, f1) in enumerate(zip(acuracias, f1_scores), 1):
    print(f"Fold {i}: Acurácia = {acc:.2f}, F1-Score = {f1:.2f}")

print("\nResultados Gerais:")
print(f"Acurácia Média: {acuracia_media:.2f}, Desvio Padrão: {acuracia_desvio:.2f}")
print(f"F1-Score Média: {f1_media:.2f}, Desvio Padrão: {f1_desvio:.2f}")

Resultados por Fold:
Fold 1: Acurácia = 0.75, F1-Score = 0.65
Fold 2: Acurácia = 0.75, F1-Score = 0.65
Fold 3: Acurácia = 0.75, F1-Score = 0.67
Fold 4: Acurácia = 1.00, F1-Score = 1.00
Fold 5: Acurácia = 0.67, F1-Score = 0.56
Fold 6: Acurácia = 1.00, F1-Score = 1.00
Fold 7: Acurácia = 0.67, F1-Score = 0.56
Fold 8: Acurácia = 0.67, F1-Score = 0.56
Fold 9: Acurácia = 1.00, F1-Score = 1.00
Fold 10: Acurácia = 0.33, F1-Score = 0.33

Resultados Gerais:
Acurácia Média: 0.76, Desvio Padrão: 0.20
F1-Score Média: 0.70, Desvio Padrão: 0.22


In [124]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score

# Criando o modelo base
modelo = DecisionTreeClassifier()

# Configuração de hiperparâmetros para teste
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random']
}

# Métricas para otimização
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_weighted': make_scorer(f1_score, average='weighted'),
}

# Configurando o GridSearchCV com validação cruzada
grid_search = GridSearchCV(
    estimator=modelo,
    param_grid=param_grid,
    scoring=scoring,
    refit='accuracy',
    cv=10,  # Validação cruzada estratificada com 10 folds
    verbose=2,
    n_jobs=-1,
)

# Treinando o modelo com GridSearch
grid_search.fit(X, classes)

# Exibindo os melhores hiperparâmetros e resultados
print("Melhores hiperparâmetros:", grid_search.best_params_)
print(f"Melhor acurácia: {grid_search.best_score_:.2f}")

# Obtendo os resultados detalhados
resultados = grid_search.cv_results_

# Iterar sobre cada combinação de hiperparâmetros
for i in range(len(resultados['params'])):
    print(f"Configuração de Hiperparâmetros {i+1}: {resultados['params'][i]}")

    # Métricas para cada fold
    for fold in range(10):  # 10 folds
        acuracia_fold = resultados[f'split{fold}_test_accuracy'][i]
        f1_fold = resultados[f'split{fold}_test_f1_weighted'][i]
        print(f"  Fold {fold+1}: Acurácia = {acuracia_fold:.2f}, F1-score = {f1_fold:.2f}")

    # Métricas médias
    acuracia_media = resultados['mean_test_accuracy'][i]
    f1_media = resultados['mean_test_f1_weighted'][i]
    acuracia_std = resultados['std_test_accuracy'][i]
    f1_std = resultados['std_test_f1_weighted'][i]

    print(f"  Acurácia média: {acuracia_media:.2f}, Desvio padrão: {acuracia_std:.2f}")
    print(f"  F1-score médio: {f1_media:.2f}, Desvio padrão: {f1_std:.2f}")
    print("-" * 50)

Fitting 10 folds for each of 4 candidates, totalling 40 fits
Melhores hiperparâmetros: {'criterion': 'entropy', 'splitter': 'best'}
Melhor acurácia: 0.80
Configuração de Hiperparâmetros 1: {'criterion': 'gini', 'splitter': 'best'}
  Fold 1: Acurácia = 0.75, F1-score = 0.65
  Fold 2: Acurácia = 0.50, F1-score = 0.38
  Fold 3: Acurácia = 0.75, F1-score = 0.67
  Fold 4: Acurácia = 1.00, F1-score = 1.00
  Fold 5: Acurácia = 0.67, F1-score = 0.56
  Fold 6: Acurácia = 0.67, F1-score = 0.56
  Fold 7: Acurácia = 0.67, F1-score = 0.56
  Fold 8: Acurácia = 1.00, F1-score = 1.00
  Fold 9: Acurácia = 1.00, F1-score = 1.00
  Fold 10: Acurácia = 0.67, F1-score = 0.56
  Acurácia média: 0.77, Desvio padrão: 0.17
  F1-score médio: 0.69, Desvio padrão: 0.21
--------------------------------------------------
Configuração de Hiperparâmetros 2: {'criterion': 'gini', 'splitter': 'random'}
  Fold 1: Acurácia = 0.50, F1-score = 0.50
  Fold 2: Acurácia = 0.50, F1-score = 0.50
  Fold 3: Acurácia = 0.75, F1-scor

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Criando o modelo base
modelo = KNeighborsClassifier()

# Configuração de hiperparâmetros para teste
param_grid = {
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree']
}

# Métricas para otimização
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_weighted': make_scorer(f1_score, average='weighted'),
}

# Configurando o GridSearchCV com validação cruzada
grid_search = GridSearchCV(
    estimator=modelo,
    param_grid=param_grid,
    scoring=scoring,
    refit='accuracy',
    cv=10,  # Validação cruzada estratificada com 10 folds
    verbose=2,
    n_jobs=-1,
)

# Treinando o modelo com GridSearch
grid_search.fit(X, classes)

# Exibindo os melhores hiperparâmetros e resultados
print("Melhores hiperparâmetros:", grid_search.best_params_)
print(f"Melhor acurácia: {grid_search.best_score_:.2f}")

# Obtendo os resultados detalhados
resultados = grid_search.cv_results_

# Iterar sobre cada combinação de hiperparâmetros
for i in range(len(resultados['params'])):
    print(f"Configuração de Hiperparâmetros {i+1}: {resultados['params'][i]}")

    # Métricas para cada fold
    for fold in range(10):  # 10 folds
        acuracia_fold = resultados[f'split{fold}_test_accuracy'][i]
        f1_fold = resultados[f'split{fold}_test_f1_weighted'][i]
        print(f"  Fold {fold+1}: Acurácia = {acuracia_fold:.2f}, F1-score = {f1_fold:.2f}")

    # Métricas médias
    acuracia_media = resultados['mean_test_accuracy'][i]
    f1_media = resultados['mean_test_f1_weighted'][i]
    acuracia_std = resultados['std_test_accuracy'][i]
    f1_std = resultados['std_test_f1_weighted'][i]

    print(f"  Acurácia média: {acuracia_media:.2f}, Desvio padrão: {acuracia_std:.2f}")
    print(f"  F1-score médio: {f1_media:.2f}, Desvio padrão: {f1_std:.2f}")
    print("-" * 50)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

# Criando o modelo base
modelo = MultinomialNB()

# Configuração de hiperparâmetros para teste
param_grid = {
    'alpha': [0.1, 1.0],
    'force_alpha': [True, False]
}

# Métricas para otimização
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_weighted': make_scorer(f1_score, average='weighted'),
}

# Configurando o GridSearchCV com validação cruzada
grid_search = GridSearchCV(
    estimator=modelo,
    param_grid=param_grid,
    scoring=scoring,
    refit='accuracy',
    cv=10,  # Validação cruzada estratificada com 10 folds
    verbose=2,
    n_jobs=-1,
)

# Treinando o modelo com GridSearch
grid_search.fit(X, classes)

# Exibindo os melhores hiperparâmetros e resultados
print("Melhores hiperparâmetros:", grid_search.best_params_)
print(f"Melhor acurácia: {grid_search.best_score_:.2f}")

# Obtendo os resultados detalhados
resultados = grid_search.cv_results_

# Iterar sobre cada combinação de hiperparâmetros
for i in range(len(resultados['params'])):
    print(f"Configuração de Hiperparâmetros {i+1}: {resultados['params'][i]}")

    # Métricas para cada fold
    for fold in range(10):  # 10 folds
        acuracia_fold = resultados[f'split{fold}_test_accuracy'][i]
        f1_fold = resultados[f'split{fold}_test_f1_weighted'][i]
        print(f"  Fold {fold+1}: Acurácia = {acuracia_fold:.2f}, F1-score = {f1_fold:.2f}")

    # Métricas médias
    acuracia_media = resultados['mean_test_accuracy'][i]
    f1_media = resultados['mean_test_f1_weighted'][i]
    acuracia_std = resultados['std_test_accuracy'][i]
    f1_std = resultados['std_test_f1_weighted'][i]

    print(f"  Acurácia média: {acuracia_media:.2f}, Desvio padrão: {acuracia_std:.2f}")
    print(f"  F1-score médio: {f1_media:.2f}, Desvio padrão: {f1_std:.2f}")
    print("-" * 50)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Criando o modelo base
modelo = LogisticRegression()

# Configuração de hiperparâmetros para teste
param_grid = {
    'solver': ['lbfgs', 'saga'],
    'class_weight': [None, 'balanced'],
}

# Métricas para otimização
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_weighted': make_scorer(f1_score, average='weighted'),
}

# Configurando o GridSearchCV com validação cruzada
grid_search = GridSearchCV(
    estimator=modelo,
    param_grid=param_grid,
    scoring=scoring,
    refit='accuracy',
    cv=10,  # Validação cruzada estratificada com 10 folds
    verbose=2,
    n_jobs=-1,
)

# Treinando o modelo com GridSearch
grid_search.fit(X, classes)

# Exibindo os melhores hiperparâmetros e resultados
print("Melhores hiperparâmetros:", grid_search.best_params_)
print(f"Melhor acurácia: {grid_search.best_score_:.2f}")

# Obtendo os resultados detalhados
resultados = grid_search.cv_results_

# Iterar sobre cada combinação de hiperparâmetros
for i in range(len(resultados['params'])):
    print(f"Configuração de Hiperparâmetros {i+1}: {resultados['params'][i]}")

    # Métricas para cada fold
    for fold in range(10):  # 10 folds
        acuracia_fold = resultados[f'split{fold}_test_accuracy'][i]
        f1_fold = resultados[f'split{fold}_test_f1_weighted'][i]
        print(f"  Fold {fold+1}: Acurácia = {acuracia_fold:.2f}, F1-score = {f1_fold:.2f}")

    # Métricas médias
    acuracia_media = resultados['mean_test_accuracy'][i]
    f1_media = resultados['mean_test_f1_weighted'][i]
    acuracia_std = resultados['std_test_accuracy'][i]
    f1_std = resultados['std_test_f1_weighted'][i]

    print(f"  Acurácia média: {acuracia_media:.2f}, Desvio padrão: {acuracia_std:.2f}")
    print(f"  F1-score médio: {f1_media:.2f}, Desvio padrão: {f1_std:.2f}")
    print("-" * 50)

In [115]:
print(type(X), X.shape)
print(type(classes), len(classes))


<class 'numpy.ndarray'> ()
<class 'numpy.ndarray'> 33


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV

# Criando o modelo base
modelo = MLPClassifier(max_iter=10)

# Configuração de hiperparâmetros para teste
param_grid = {
    'max_iter': [10, 200],
    'activation': ['relu', 'tanh']
}

# Métricas para otimização
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'f1_weighted': make_scorer(f1_score, average='weighted'),
}

# Configurando o GridSearchCV com validação cruzada
grid_search = GridSearchCV(
    estimator=modelo,
    param_grid=param_grid,
    scoring=scoring,
    refit='accuracy',
    cv=10,  # Validação cruzada estratificada com 10 folds
    verbose=2,
    n_jobs=-1,
)

# Treinando o modelo com GridSearch
grid_search.fit(X, classes)

# Exibindo os melhores hiperparâmetros e resultados
print("Melhores hiperparâmetros:", grid_search.best_params_)
print(f"Melhor acurácia: {grid_search.best_score_:.2f}")

# Obtendo os resultados detalhados
resultados = grid_search.cv_results_

# Iterar sobre cada combinação de hiperparâmetros
for i in range(len(resultados['params'])):
    print(f"Configuração de Hiperparâmetros {i+1}: {resultados['params'][i]}")

    # Métricas para cada fold
    for fold in range(10):  # 10 folds
        acuracia_fold = resultados[f'split{fold}_test_accuracy'][i]
        f1_fold = resultados[f'split{fold}_test_f1_weighted'][i]
        print(f"  Fold {fold+1}: Acurácia = {acuracia_fold:.2f}, F1-score = {f1_fold:.2f}")

    # Métricas médias
    acuracia_media = resultados['mean_test_accuracy'][i]
    f1_media = resultados['mean_test_f1_weighted'][i]
    acuracia_std = resultados['std_test_accuracy'][i]
    f1_std = resultados['std_test_f1_weighted'][i]

    print(f"  Acurácia média: {acuracia_media:.2f}, Desvio padrão: {acuracia_std:.2f}")
    print(f"  F1-score médio: {f1_media:.2f}, Desvio padrão: {f1_std:.2f}")
    print("-" * 50)


Fitting 10 folds for each of 4 candidates, totalling 40 fits
Melhores hiperparâmetros: {'activation': 'tanh', 'hidden_layer_sizes': (100,)}
Melhor acurácia: 0.97
Configuração de Hiperparâmetros 1: {'activation': 'relu', 'hidden_layer_sizes': (50,)}
  Fold 1: Acurácia = 0.75, F1-score = 0.75
  Fold 2: Acurácia = 1.00, F1-score = 1.00
  Fold 3: Acurácia = 0.75, F1-score = 0.67
  Fold 4: Acurácia = 1.00, F1-score = 1.00
  Fold 5: Acurácia = 0.67, F1-score = 0.56
  Fold 6: Acurácia = 0.67, F1-score = 0.56
  Fold 7: Acurácia = 1.00, F1-score = 1.00
  Fold 8: Acurácia = 1.00, F1-score = 1.00
  Fold 9: Acurácia = 1.00, F1-score = 1.00
  Fold 10: Acurácia = 0.33, F1-score = 0.17
  Acurácia média: 0.82, Desvio padrão: 0.21
  F1-score médio: 0.77, Desvio padrão: 0.27
--------------------------------------------------
Configuração de Hiperparâmetros 2: {'activation': 'relu', 'hidden_layer_sizes': (100,)}
  Fold 1: Acurácia = 0.75, F1-score = 0.75
  Fold 2: Acurácia = 1.00, F1-score = 1.00
  Fold 

d:\Arquivos\EngSoft\IA\trab\bow\.env\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(
